In [1]:
# importing all necessary modules 
import warnings  
import gensim 
import utils
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize 
from gensim.models import Word2Vec 
from nltk.corpus import stopwords 

warnings.filterwarnings(action = 'ignore')

# https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/
# https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92#9731

In [2]:
train = utils.load_dataset('datasets/Wikidata/lcquad2_anstype_wikidata_train.json')
test = utils.load_dataset('datasets/Wikidata/lcquad2_anstype_wikidata_test_gold.json')

In [3]:
stop_words = set(stopwords.words('english'))

lines = []
for doc in train:
    if doc['category'] == 'resource':
        q = doc['question']
        q = " ".join([w for w in word_tokenize(q)  if not w in stop_words])
        lines.append(q + " " + " ".join(doc['type']))
        
data = [] 

for line in lines:
    temp = []
    for j in word_tokenize(line): 
        temp.append(j.lower()) 
    data.append(temp)

In [4]:
# Create CBOW model 
model1 = gensim.models.Word2Vec(data, min_count = 1,size = 100, window = 10) 
  
# Create Skip Gram model 
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100,window = 10, sg = 1) 

In [5]:
pickle.dump(model2, open('word2vec_sg.sav', 'wb'))

In [6]:
word1 = 'delta'
word2 = 'publication'
print("CBOW : ", model1.similarity(word1, word2))
print("Skip Gram : ", model2.similarity(word1, word2))

CBOW :  0.913829
Skip Gram :  0.7495525


In [11]:
model2.most_similar('cola')

[('carbonated', 0.9915328025817871),
 ('non-alcoholic', 0.988882303237915),
 ('soft', 0.9813560843467712),
 ('drink', 0.9802319407463074),
 ('airliner', 0.973507285118103),
 ('equipment', 0.9714939594268799),
 ('beverage', 0.9703865051269531),
 ('fraction', 0.967510461807251),
 ('low-cost', 0.96715247631073),
 ('motorcycle', 0.9666259288787842)]

In [12]:
q_terms = ['what', 'periodical', 'literature', 'delta', 'air', 'lines', 'use', 'moutpiece']
t_terms = ['publication', 'recurring', 'text', 'serial', 'airport', 'aerodrome', 'station', 'class', 'product']
# q_terms = ['who', 'country', 'head', 'state', 'mahmoud', 'abbas']
# t_terms = ['regime', 'state', 'country', 'person', 'omnivore', 'nation', 'republic', 'plutocracy']

for i in q_terms:
    for j in t_terms:
        score = model2.similarity(i, j)
        if score > 0.85:
            print(i, j, score)

periodical publication 0.9086688
literature text 0.9018323
delta aerodrome 0.9145688
lines aerodrome 0.8962494
moutpiece serial 0.87807333


In [9]:
for i in q_terms:
    for j in t_terms:
        score = model1.similarity(i, j)
        if score > 0.95:
            print(i, j, score)

country state 0.9835948
country country 1.0
country republic 0.9667765
state state 1.0
state country 0.9835948
state republic 0.98351127
mahmoud regime 0.95115507
